In [ ]:
import sys; sys.path.append("..")
#from os.path import abspath; print(abspath('.'))
from utils import count_down

# Current Homework

![homework: factory method](hw_factory.png)

There are two errors in the following solution, what are they?

In [ ]:
@staticmethod
def filled(self, rows, cols, value):
    sample = []
    for i in range(cols):
        sample.append(value)
    result = []
    for i in range(rows):
        result.append(sample)
    self._matrix = result
    
count_down(3)

In [ ]:
rows = cols = 3; value = 2
sample = []
for i in range(cols):
    sample.append(value)
result = []
for i in range(rows):
    result.append(sample)
    
result

In [ ]:
result[0][1] = 3
result

* The 'filled'-method is a factory-method, which are always static. Static methods are not called on instances of objects and don't know specific instances, therefore using the keyword ``self`` is wrong. Instead, a temporary matrix should be created and then returned using the class's own constructor
* The first row of the matrix (a list-object) is generated and then used over and over again. This means the very same object gets re-used for all subsequent rows - which means that if you change an element of the first row, you change it for all rows.

# Numerical Computing with NumPy

## Introductory example

Python built-in collections like `list` offer a flexible way of storing and maniupulating data. As dicussed previously, collections usually just store references to objects. While this is every convenient when writing code, it comes with costs in performance in memory. 

Let's look at an example. Say we took one million measurements in an experiment and now want to compute the mean of it. We could do it in the following manner. 

In [ ]:
import random 
measurements = [random.randint(150, 200) for _ in range(1_000_000)]
measurements

In [ ]:
import random 
measurements = [random.randint(150, 200) for _ in range(1_000_000)]

def calculate_mean(measurements):
    accumulator = 0
    for measurement in measurements:
        accumulator += measurement
    
    mean = accumulator / len(measurements)
    return mean

%timeit calculate_mean(measurements)

This is rather slow since Python has to rebind a new variable in every loop and then has to check whether the `+` operation is supported between the `accumulator` and the current `measurement`. This prevents it from trying to add together objects that can't be added, but in this case we are pretty sure that we are only dealing with integers. If we could tell the interpreter that we are only adding integers, we could skip all that typechecking and speed up the operation. For this purpose, `numpy` was invented.  


To use numpy we have to import it. The import is usuall aliased as `np` so we have to type less later on. Aliasing things is only recommended if it is well established in the community of the respective package.

In [ ]:
import numpy as np

Numpy's standard datatype is the `ndarray` (which stands for n-dimensional array). In the simplest case, numpy array can be created from list.

In [ ]:
measurements_array = np.array(measurements)
measurements_array

In [ ]:
type(measurements_array)

They behave very similar to list, but have a fixed datatype underneath. Numpy automatically notices that all our values are intergers and chooses the appropriate datatype. An integer that takes up 64 bits of memory. https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html

In [ ]:
measurements_array[0]

In [ ]:
measurements_array[0:5]

In [ ]:
measurements_array.dtype

Moreover, numpy offers a lot of routines for mathematical operations of arrays. Let's see if we acually gained something by using numpy.

In [ ]:
%timeit np.mean(measurements_array)

Almost 100x speedup in comparision to the pure Python implementation! After convincing ourselfs that NumPy is useful, we have a more in depth look at the numpy array.

## Anatomy of arrays
Every array has a bunch of attributes that yield inforation about what it is.

### dtype

`.dtype` gives information about the data type. arrays can contain bools, ints, unsigned ints, floats or complex numbers of various byte sizes. They can also store strings or Python objects, but that has very few use cases.

In [ ]:
values = [0, 1, 2, 3, 4]
int_arr = np.array(values, dtype='int')
int_arr, int_arr.dtype

If the dtype does not match the given values, numpy will cast everything to that data type.

In [ ]:
bool_arr = np.array(values, dtype='bool')
bool_arr, bool_arr.dtype

If no explicit data type is given, numpy will choose the "smallest common denominator". In the following example, everything becomes a float, as ints can be represented as floats, but not vice versa.

In [ ]:
values = [0, 1, 2.5, 3, 4]
float_arr = np.array(values)
float_arr, float_arr.dtype

However, once the data type is set, everything will be coerced to that type.

In [ ]:
int_arr[1] = 2.5
int_arr, int_arr.dtype

These non-Python data types force us to again think about problems like overflow etc.

In [ ]:
values = [0, 1, 2, 3, 4]
uint_arr = np.array(values, dtype='int8')
uint_arr, uint_arr.dtype

In [ ]:
uint_arr[1] += 255
uint_arr

...and can lead to some problems when comparing them to standard python types

In [ ]:
type(measurements_array[0]) == type(183)

In [ ]:
np.array([1.2-1.0], dtype=np.float32)[0] == 1.2-1.0

For better comparisons, you can compare using an epsilon-value:

In [ ]:
epsilon = 0.00001
abs(np.array([1.2-1.0], dtype=np.float32)[0] - (1.2-1.0) ) < epsilon

http://effbot.org/pyfaq/why-are-floating-point-calculations-so-inaccurate.htm

### shape and ndim
`.shape` is very important for keeping track of arrays with more than one dimension. It is a tuple with the number of elementns in each dimension. `.ndim` is just the number of dimensions in total. 

In [ ]:
values = [0, 1, 2, 3, 4]
one_dim_arr = np.array(values)
one_dim_arr

In [ ]:
one_dim_arr.shape

In [ ]:
one_dim_arr.ndim

In [ ]:
values = [[0, 1, 2, 3, 4]] * 3
two_dim_arr = np.array(values)
two_dim_arr

In [ ]:
[[0, 1, 2, 3, 4]] * 3

In [ ]:
two_dim_arr.shape

In [ ]:
two_dim_arr.ndim

In [ ]:
two_dim_arr[1,1] = 10

In [ ]:
two_dim_arr

In [ ]:
values = [[[0, 1, 2, 3, 4]] * 3] * 6
three_dim_arr = np.array(values)
three_dim_arr

In [ ]:
three_dim_arr.shape

In [ ]:
three_dim_arr.ndim

### Other attributes

In [ ]:
two_dim_arr

In [ ]:
two_dim_arr.T

In [ ]:
print(dir(two_dim_arr))

## Creating arrays
We already saw how arrays can be created from Python lists (the same works with tuples). However, we often would like to create arrays directly, without creating Python objects. This can be accomplished by several utility functions.

The equivalent of `range`.

In [ ]:
np.arange(9)

In [ ]:
np.arange(start=2, stop=14, step=2)

Creating an array with a certain number of values in a certain interval.

In [ ]:
np.linspace(start=-5, stop=5, num=10)

An array containing zeros. The default `dtype` is `float`.

In [ ]:
np.zeros(5)

`np.zeros` takes a `shape` argument that lets us create multidimensional arrays.

In [ ]:
np.zeros((2, 3, 2))

The same goes for `ones`, `empty` and `full`.

In [ ]:
np.ones(shape=(2, 3, 2))

In [ ]:
# Corresponds to whatever was left in memory. Using zeros for initialising arrays is usually saver.
np.empty(shape=(2, 3, 2))

In [ ]:
np.full(shape=(2, 3, 2), fill_value=42)

### Exercise

Create a 3*3 array that solely consists of True

In [ ]:
count_down(2)

In [ ]:
np.ones((3,3), dtype=bool)

`np.random` contains a lot of functions to create arrays filled with random values of various probability distributions.

In [ ]:
np.random.random((3, 3))

In [ ]:
np.random.randint(0, 10, (5, 5))

Using ``np.random.randint`` and a boolean dtype, you can create random boolean arrays!

### Exercise
 
Create a 5x5 array in which *statistically* 1/4 of items are False, all others being True.

In [ ]:
count_down(3)

You can use the method ``astype(bool)`` to convert an integer-array into a boolean-array

In [ ]:
np.random.randint(0, 4, (5, 5)).astype(bool)

Note that in this array, there are only *on average* 1/4 elements of the elements True. To create an array where precisely 1/4 of elements are True, one would create an array with that many True's and ``shuffle`` it, as we'll see lateron.

``np.repeat`` repeats elements of an array:

In [ ]:
np.repeat(3, 5)

In [ ]:
np.repeat([[1,2],[3,4]], 2)

### Reshape

In [ ]:
a = np.arange(start=2, stop=14)
print(a.shape)
a

In [ ]:
b = a.reshape(3, 4)
b

-1 as axis automatically figures out the size of the respective dimension

In [ ]:
a.reshape(-1, 2)

### Exercise

Using only what you've seen so far and only using numpy, create a two-dimensional array that contains the sequence [1, 2, 3] in every row (10 rows)

In [ ]:
count_down(5)

In [ ]:
# what you're supposed to create:
np.array([1, 2, 3]*10).reshape(10, -1)
# however, numpy doesn't allow the *10-syntax the way lists do (and this leads to the aforementioned side effects)

The transpose of the matrix can help you greatly!

In [ ]:
np.repeat(np.arange(1, 4), 10).reshape(3, -1).T

### Comparing Arrays

In [ ]:
epsilon = 0.000000000001
a = np.zeros((3,3))
b = np.zeros((3,3))
a[0,0] += 0.5*epsilon
a == b

In [ ]:
(a == b).all()

flaws with doing this: 
* if either a or b is empty and the other one contains a single element, this will return True. (the comparison a==b returns an empty array, for which the all-operator returns True)
* If a and b don't have the same shape and aren't broadcastable, then this approach will raise an error.

Instead, use numpys provided functions!

In [ ]:
np.array_equal(a, b)

In [ ]:
np.allclose(a, b)

In [ ]:
np.isclose(a, b)

## Masking
Logical arrays, i.e. arrays containing boolean values, can be used to index other arrays. These logical arrays are then called masks. This is especially useful to index based on logical conditions.

In [ ]:
# A simple integer array.
arr = np.arange(1, 6)
arr

In [ ]:
# A boolean array of the same shape as arr.
mask = np.array([True, False, True, False, True])
mask

Using the mask for indexing returns an array with only elements at positions where `mask` is `True`.  

In [ ]:
arr[mask]

Luckily for us, Operators in numpy work element-wise and return a boolean array:

In [ ]:
arr < 10

Because of this, we can use direct comparison as a mask:

In [ ]:
arr[arr < 10]

Maks can be used for assignment, which keeps the shape of the original array (thanks to *fancy indexing*, which will come up later)

In [ ]:
arr[mask] = 10
arr

### Exercise

Replace all odd numbers in the given array with -1

In [ ]:
count_down(3)

In [ ]:
arr = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
arr

In [ ]:
arr[arr % 2 == 1] = -1
arr

## Mathematical operations
Numpy contains a lot of mathematical functions that operate on arrays in a vectorized manner. That means that they are applied to each element, without explicit for-loops. Vectorized functions are called `ufuncs` (universal functions) in Numpy.

### Standard arithmetic

In [ ]:
arr = np.arange(9)
arr

In [ ]:
arr * 3

In [ ]:
arr + (arr*2)

In [ ]:
arr - arr

In [ ]:
1 /0

In [ ]:
arr / arr

In [ ]:
arr * arr

In [ ]:
arr ** 2

### Exercise
Create a 1-dimensional array that repeats the sequence [1, 2, 3] 30 times!

In [ ]:
count_down(3)

In [ ]:
a = np.arange(3*30) % 3 + 1
a

In [ ]:
var1 = lambda: np.repeat(np.arange(1, 4), 30).reshape(3, -1).T.flatten()
var2 = lambda: np.arange(3*30) % 3 + 1
var3 = lambda: np.array([[1,2,3] for _ in range(30)]).flatten()

%timeit var1()
%timeit var2()
%timeit var3()

Using `@` you can even do matrix multiplication. In the case of 1d arrays, this is the inner product between two vectors.

In [ ]:
arr @ arr

In [ ]:
# That's the same as
np.sum(arr * arr)

### Some standard functions

In [ ]:
arr

In [ ]:
np.log(arr)

In [ ]:
np.exp(arr)

In [ ]:
np.sin(arr)

np.sign returns -1 for negative values, +1 for positive ones, and 0 for 0:

In [ ]:
np.sign(arr)

Always try to use vectorized ufuncs instead of explicit loops!

### Broadcasting
What happens if you try to add arrays of different shapes? Numpy will try to expand the arrays according to three rules and try to make their shapes match, so the operation can be applied elementwise. 

**1. Rule** If the arrays have different numbers of dimensions, the smaller shape is padded with ones on its left side.<br/>
            Example: (5 x 3) + (3) &rarr; (5 x 3) + (**1** x 3)<br/>
**2. Rule** If the number of the dimensions matches, but the size of a dimension does not, dimensions with the size of 1 are expanded.<br/>
            Example: (5 x 3) + (1 x 3) &rarr; (5 x 3) + (**5** x 3)<br/>
**3. Rule** If the shapes of the  arrays still defer after applying the Rule 1 and 2, a broadcasting error is raised.

The figure below gives an illustration (source https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html) ![](broadcasting.png)




The Numpy documentation gives further insights https://docs.scipy.org/doc/numpy-1.14.0/user/basics.broadcasting.html. 


In [ ]:
a = np.arange(15).reshape((5, 3))
a

In [ ]:
b = np.arange(3)
b

In [ ]:
a + b 

Here is a case in which broadcasting fails.

In [ ]:
c = np.arange(4)
a + c

### Aggregations functions
Aggregation function are functions that reduce the dimensionality of an array. They provide an `axis` argument, to specify which dimension to reduce.

In [ ]:
np.random.seed(1)
two_dim_arr = np.random.randint(0, high=20, size=(4, 4))
two_dim_arr

If just the array is passed, the aggregation operation is performed over the whole array.

In [ ]:
np.min(two_dim_arr)

The optional `axis` argument allows us to specify, which dimension should be aggregated. You can think of it as the operation being applied to all entries that are obtained by keeping the indices in all dimensions fixed except for the `axis` dimension.
Let's look at the result of the minimum operation with `axis=0`:

In [ ]:
np.min(two_dim_arr, axis=0)

The axis concept extends to more than one dimension

In [ ]:
np.random.seed(1)
three_dim_arr = np.random.randint(0, high=20, size=(4, 4, 4))
three_dim_arr

In [ ]:
np.min(three_dim_arr, axis=0)

Here the entry at index `[0, 0]`, i.e. `5` is the minimum of the following values. 

In [ ]:
for i in range(4):
    print(three_dim_arr[i, 0, 0])

Let's demonstrate all axes again with another three-dimensional array:

In [ ]:
a = np.array([[[2,4],[6,9]],[[3,1],[7,8]],[[4,5],[9, 0]]])

In [ ]:
a, a.shape

In [ ]:
np.min(a)

In [ ]:
np.min(a, axis=0)

setting the axis-argument is the same as going through all other axes of the respective array in turn, returning the respective aggregate for every combination of these.

In [ ]:
for i in range(a.shape[1]):
    for j in range(a.shape[2]):
        print(a[:, i, j])

For axis=1, we loop through axis 0 and axis 2:

In [ ]:
a

In [ ]:
np.min(a, axis=1)

In [ ]:
for i in range(a.shape[0]):
    for j in range(a.shape[2]):
        print(a[i, :, j])

...and finally, for axis 2 we loop through axis 0 and 1

In [ ]:
a

In [ ]:
np.min(a, axis=2)

In [ ]:
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        print(a[i, j, :])

The shape of the resulting array is simply the shape of the original array, leaving the specified axis out:

In [ ]:
mins = []
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        mins.append(min(a[i,j,:]))
np.array(mins).reshape([a.shape[0], a.shape[1]])

In [ ]:
for ax in range(3):
    print(np.min(a, axis=ax).shape == a.shape[:ax]+a.shape[ax+1:])

...however, of course, using numpy is much faster than looping over the array:

In [ ]:
def find_min_manual(arr):
    mins = []
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            mins.append(min(arr[i,j,:]))
    np.array(mins).reshape([arr.shape[0], arr.shape[1]])

%timeit find_min_manual(a)
%timeit np.min(a, axis=2)

**End of tuesday-lecture**

**Addendum to tuesday**

**Repeating arrays**

While np.repeat does not repeat the way we would want to, np.tile does:

In [21]:
np.tile([1,2,3], 3)

array([1, 2, 3, 1, 2, 3, 1, 2, 3])

**np.nan**

In [24]:
np.nan == np.nan

False

In [25]:
np.isnan(np.nan)

True

In [64]:
a = np.r_[np.arange(5), np.repeat(0, 5)]
a

array([0, 1, 2, 3, 4, 0, 0, 0, 0, 0])

In [65]:
b = a/a
b

/home/chris/anaconda3/envs/scientific_programming/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([nan,  1.,  1.,  1.,  1., nan, nan, nan, nan, nan])

In [66]:
b[~np.isnan(b)]

array([1., 1., 1., 1.])

In [68]:
np.divide(a, a, out=np.zeros(a.shape), where=(a!=0)) 
#at the positions where a!=0, make the division, 
#at other indices use what's specified as "out"

array([0., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

### Aggregations functions
Aggregation function are functions that reduce the dimensionality of an array. They provide an `axis` argument, to specify which dimension to reduce.

In [ ]:
np.random.seed(1)
two_dim_arr = np.random.randint(0, high=20, size=(4, 4))
two_dim_arr

If just the array is passed, the aggregation operation is performed over the whole array.

In [ ]:
np.min(two_dim_arr)

The optional `axis` argument allows us to specify, which dimension should be aggregated. You can think of it as the operation being applied to all entries that are obtained by keeping the indices in all dimensions fixed except for the `axis` dimension.
Let's look at the result of the minimum operation with `axis=0`:

In [ ]:
np.min(two_dim_arr, axis=0)

The axis concept extends to more than one dimension

In [ ]:
np.random.seed(1)
three_dim_arr = np.random.randint(0, high=20, size=(4, 4, 4))
three_dim_arr

In [ ]:
np.min(three_dim_arr, axis=0)

Here the entry at index `[0, 0]`, i.e. `5` is the minimum of the following values. 

In [ ]:
for i in range(4):
    print(three_dim_arr[i, 0, 0])

#### More than one Dimension

Aggregation functions can also aggregate more than one dimension at once.

In [ ]:
np.min(three_dim_arr, axis=(1, 2))

Here the entry at index `[2]`, i.e. `3` is the minimum of the following values. 

In [ ]:
for i in range(4):
    for j in range(4):
        print(three_dim_arr[2, i, j])

#### Other aggregation functions.

In [ ]:
two_dim_arr

In [ ]:
np.max(two_dim_arr)

Let's demonstrate all axes again with another three-dimensional array:

In [ ]:
a = np.array([[[2,4],[6,9]],[[3,1],[7,8]],[[4,5],[9, 0]]])

In [ ]:
a, a.shape

In [ ]:
np.min(a)

In [ ]:
np.min(a, axis=0)

setting the axis-argument is the same as going through all other axes of the respective array in turn, returning the respective aggregate for every combination of these.

In [ ]:
for i in range(a.shape[1]):
    for j in range(a.shape[2]):
        print(a[:, i, j])

For axis=1, we loop through axis 0 and axis 2:

In [ ]:
a

In [ ]:
np.min(a, axis=1)

In [ ]:
for i in range(a.shape[0]):
    for j in range(a.shape[2]):
        print(a[i, :, j])

...and finally, for axis 2 we loop through axis 0 and 1

In [ ]:
a

In [ ]:
np.min(a, axis=2)

In [ ]:
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        print(a[i, j, :])

The shape of the resulting array is simply the shape of the original array, leaving the specified axis out:

In [ ]:
mins = []
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        mins.append(min(a[i,j,:]))
np.array(mins).reshape([a.shape[0], a.shape[1]])

In [ ]:
for ax in range(3):
    print(np.min(a, axis=ax).shape == a.shape[:ax]+a.shape[ax+1:])

...however, of course, using numpy is much faster than looping over the array:

In [ ]:
def find_min_manual(arr):
    mins = []
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            mins.append(min(arr[i,j,:]))
    np.array(mins).reshape([arr.shape[0], arr.shape[1]])

%timeit find_min_manual(a)
%timeit np.min(a, axis=2)

In [ ]:
np.max(two_dim_arr, axis=0)

In [ ]:
np.max(two_dim_arr, axis=1)

In [ ]:
np.sum(two_dim_arr)

In [ ]:
np.sum(two_dim_arr, axis=0)

In [ ]:
np.sum(two_dim_arr, axis=1)

Many of these function are also available as method on the array object.

In [ ]:
two_dim_arr.sum(axis=0)

### Exercise

Create a Lambda-function to flatten a given numpy-array, such that it is one-dimensional

In [ ]:
a = np.arange(64).reshape((2,2,2,2,2,2))
a

In [ ]:
count_down(3)

In the end, you want to create a shape of (product_of_all_dimensions, 1).

In [ ]:
flatten = lambda x: x.reshape(np.product(x.shape))

In [ ]:
flatten(a)

...of course, there's also a numpy-function for that.

In [ ]:
a.flatten()

## Advanced indexing
Numpy provides indexing methods that go beyond the indexing techniques known from standard Python sequences.


### Multidimensional indexing
Instead of doing subsequent indexing as with standard Python lists you can index all dimensions at once.

In [86]:
two_dim_list = [
    [ 0,  1,  2],
    [ 3,  4,  5],
    [ 6,  7,  8],
    [ 9, 10, 11],
    [12, 13, 14]
]
two_dim_list[2][1]

7

In [87]:
inner_list = two_dim_list[2]
inner_list[1]

7

In [88]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

7

In [ ]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

You can use a colon to get all values from that dimensions.

In [ ]:
large_two_dim_arr = np.arange(81).reshape((9, 9))
large_two_dim_arr

In [ ]:
large_two_dim_arr[:, 1]

Standard slicing with `(start, stop, step)` works as expected.

In [ ]:
large_two_dim_arr[:, 1:3]

In [ ]:
large_two_dim_arr[:, 2:7:2]

Slices of an array are always `views`. That means, you just "view" the same chunk of meomory from a different perspective. This saves a lot of memory, but it means also that the original array will be changed, if you change the view.

In [ ]:
arr_slice = large_two_dim_arr[:, 1]
arr_slice[:] = 0
large_two_dim_arr

In [ ]:
large_two_dim_arr[:, 2] =0
large_two_dim_arr

If you need all values from several consecutive dimensions you can use ellipsis (`...`) as a shorthand.

In [ ]:
# Ellipsis is an actual Python object.
print(...)

In [ ]:
# np.stack joins arrays along a new axis.
four_dim_arr = np.stack((np.ones((3, 3, 3)), 
                         np.ones((3, 3, 3)) * 2, 
                         np.ones((3, 3, 3)) * 3, 
                         np.ones((3, 3, 3)) * 4))
four_dim_arr

In [ ]:
four_dim_arr.shape

In [ ]:
four_dim_arr[3, :, :, :]

In [ ]:
four_dim_arr[1,..., 1]

In [ ]:
four_dim_arr[1, :, :, 1]

### Fancy indexing
You can pass an array containing indices, this especially useful for drawing random items from an array.

In [82]:
arr = np.arange(9) + 10
arr

array([10, 11, 12, 13, 14, 15, 16, 17, 18])

In [83]:
indices = np.array([1, 4, 5])
arr[indices]

array([11, 14, 15])

The resulting array will reflect the shape of the index array.

In [84]:
indices = np.array([[1, 4],
                    [5, 7]])
arr[indices]

array([[11, 14],
       [15, 17]])

You can index each dimension separately.

In [89]:
two_dim_arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [90]:
x_indices = np.array([3, 4])
y_indices = np.array([1, 2])
two_dim_arr[x_indices, y_indices] # Corresponds to indexing at [3, 1] and [4, 2].

array([10, 14])

You can use indexing to swap rows and columns of an array:

In [12]:
arr = np.arange(9).reshape(3,3)
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [18]:
arr[[2, 1, 0], :]

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

In [19]:
arr[::-1]

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

### Exercise
Using fancy indexing, create an array b, that is a one-hot-encoding of array **a**:

In [1]:
a = np.random.randint(0,4, size=6)
a

array([0, 1, 3, 0, 2, 2])

In [ ]:
count_down(3)

In [4]:
#let's first create an array b of thecorrect shape...
b = np.zeros((a.shape[0], np.max(a)+1))
b

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
b[np.arange(a.shape[0]), a] = 1
b

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

Using ``np.argsort``, you can sort the indices of an array, such that you can sort other arrays the same way (note that I'm using vstack only for demonstration purposes)

In [135]:
a = np.random.randint(0, 10, 10)
b = a**2
np.vstack((a, b))

array([[ 7,  2,  8,  6,  0,  5,  6,  3,  0,  8],
       [49,  4, 64, 36,  0, 25, 36,  9,  0, 64]])

In [137]:
indices = a.argsort()

In [139]:
np.vstack((a[indices], b[indices]))

array([[ 0,  0,  2,  3,  5,  6,  6,  7,  8,  8],
       [ 0,  0,  4,  9, 25, 36, 36, 49, 64, 64]])

### Advanced Masking

In [ ]:
a = np.arange(9)
a

In [ ]:
a % 3 == 0

In [ ]:
a[a % 3 == 0]

Assigning values using masking only works because the result of an applied mask-operation is a list, so this is in the end fancy indexing!

In [ ]:
a[a % 3 == 0] = 0
a

In [ ]:
a[np.array([0, 3, 6])] = 10
a

In [ ]:
We can use masking to ex 

In [21]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
iris = np.genfromtxt(url, delimiter=',', dtype='object')
np.vstack((iris[:3], iris[-3:])) #first and last three, as demonstration

array([[b'5.1', b'3.5', b'1.4', b'0.2', b'Iris-setosa'],
       [b'4.9', b'3.0', b'1.4', b'0.2', b'Iris-setosa'],
       [b'4.7', b'3.2', b'1.3', b'0.2', b'Iris-setosa'],
       [b'6.5', b'3.0', b'5.2', b'2.0', b'Iris-virginica'],
       [b'6.2', b'3.4', b'5.4', b'2.3', b'Iris-virginica'],
       [b'5.9', b'3.0', b'5.1', b'1.8', b'Iris-virginica']], dtype=object)

In [23]:
numeric_column = iris[:, 1].astype('float')  # sepalwidth
grouping_column = iris[:, 4]  # species

output = []
for group_val in np.unique(grouping_column):
    output.append([group_val, numeric_column[grouping_column==group_val].mean()])

output

[[b'Iris-setosa', 3.418],
 [b'Iris-versicolor', 2.7700000000000005],
 [b'Iris-virginica', 2.974]]

### Exercise

Make a numpy array see the positive side by replacing all its negative values by the mean of its positive values. 

In [7]:
arr = np.random.uniform(-5, 5, 10)
arr

array([ 1.5750258 , -1.19150466, -1.55032072, -1.75232586, -2.73136747,
       -3.91308761, -3.91785124,  3.99173033, -3.24288346,  2.51980372])

In [ ]:
count_down(2)

In [9]:
arr2 = np.copy(arr)
arr2[arr2 < 0] = arr2[arr2 > 0].mean()
arr2 

array([1.5750258 , 2.69551995, 2.69551995, 2.69551995, 2.69551995,
       2.69551995, 2.69551995, 3.99173033, 2.69551995, 2.51980372])

#### Combining masks

Mask can be created by using logical operators. For example, to get all the entries in an array that are greater than two.


In [ ]:
arr = np.arange(1, 6)
greater_two = arr > 2
greater_two


In [ ]:
arr[greater_two]

Or even shorter.

In [ ]:
arr[arr > 2]

Different masks can be combined using bitwise logical operators. These are the vectorized version of locial operators and should not be confounded with `and`, `or` and `not` with try to evaluated the truth value of a whole object.

In [ ]:
smaller_or_equal_four = arr <= 4
smaller_or_equal_four   

Bitwise and `&`.

In [ ]:
greater_two & smaller_or_equal_four

In [ ]:
# This does not work.
greater_two and smaller_or_equal_four

In [ ]:
arr[greater_two & smaller_or_equal_four]

Bitwise or using `|`.

In [ ]:
arr[greater_two | smaller_or_equal_four]

Bitwise xor using `^`.

In [ ]:
arr[greater_two ^ smaller_or_equal_four]

Bitwise negation using `~`.

In [ ]:
# Gives everything smaller or equal to 2.
arr[~greater_two]

#### Using np.where

Using masking always changes the original array, whereas sometimes the original array should rather be unchanged. ``np.where`` figures out the indices of an array where the given condition is true.

In [93]:
a = np.arange(9).reshape(3, 3)
a[a % 3 == 0] = 0
a

array([[0, 1, 2],
       [0, 4, 5],
       [0, 7, 8]])

In [94]:
a = np.arange(9).reshape(3, 3)
tmp = np.where(a % 3 == 0)
tmp

(array([0, 1, 2]), array([0, 0, 0]))

In [95]:
b = np.ones((3, 3))
b[tmp] = 0
b

array([[0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.]])

``where`` can also be used to assign values to a new array:

In [96]:
np.where(a % 3 == 0, 0, a)

array([[0, 1, 2],
       [0, 4, 5],
       [0, 7, 8]])

``np.argwhere`` returns the indices grouped by element:

In [131]:
a = np.eye(4)*np.arange(16).reshape(4,4)
a

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  5.,  0.,  0.],
       [ 0.,  0., 10.,  0.],
       [ 0.,  0.,  0., 15.]])

In [132]:
np.argwhere(a)

array([[1, 1],
       [2, 2],
       [3, 3]])

In [133]:
np.where(a)

(array([1, 2, 3]), array([1, 2, 3]))

### Exercise

Create an array of 8 8-dimensional one-hot-vectors

In [ ]:
count_down(5)

In [80]:
np.array(np.arange(8*8).reshape((8, 8)) % 9 == 0, dtype=int)

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
np.eye(8, dtype=int)

### Exercise

Create a new array in which all odd numbers of ``a`` are replaced with -1 without changing arr

In [ ]:
a = np.arange(10)
a

In [ ]:
count_down(3)

In [ ]:
np.where(a % 2 == 1, -1, a)

### Exercise

Figure out the common items of the arrays a and b, where the arrays match:

In [ ]:
a = np.array([1,2,3,2,3,4,3,4,5,6])
b = np.array([7,2,10,2,7,4,9,4,9,8])

In [ ]:
count_down(2)

In [ ]:
np.unique(a[np.where(a == b)])

In [ ]:
np.intersect1d(a,b)

## Extending arrays

### Adding new dimensions with `np.newaxis`

Instead of `np.newaxis`, `None` can be used.

In [81]:
one_dim_arr = np.arange(5)
one_dim_arr, one_dim_arr.shape

(array([0, 1, 2, 3, 4]), (5,))

In [82]:
two_dim_arr = one_dim_arr[np.newaxis, :]
two_dim_arr, two_dim_arr.shape

(array([[0, 1, 2, 3, 4]]), (1, 5))

In [ ]:
two_dim_arr = one_dim_arr[:, np.newaxis, None]
two_dim_arr, two_dim_arr.shape

Adding new dimensions is useful for example when Tensorflow is used to batch-inputs, but you want to provide a single datapoint for prediction:

In [ ]:
one_dim_arr[:, None]

### Removing dimensions

``arr.squeeze()`` removes dimensions of size 1:

In [84]:
one_dim_arr = np.arange(5)
two_dim_arr = one_dim_arr[np.newaxis, :]
two_dim_arr, two_dim_arr.shape

(array([[0, 1, 2, 3, 4]]), (1, 5))

In [87]:
two_dim_arr.squeeze(), two_dim_arr.squeeze().shape

(array([0, 1, 2, 3, 4]), (5,))

In [91]:
a = np.arange(5).reshape(1, -1, 1, 1)
a

array([[[[0]],

        [[1]],

        [[2]],

        [[3]],

        [[4]]]])

In [92]:
a.squeeze()

array([0, 1, 2, 3, 4])

### Combining arrays
There are many ways to combine existing arrays, like `np.append`, `np.concatenate` and `np.stack`. However, these operations always require the whole array to be copied. Therefore, it often makes more sense to allocate an array of the size you need later upfront and then just fill the respective parts.

In [30]:
np.arange(10), np.arange(10)[1::-1]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([1, 0]))

In [31]:
np.concatenate((np.arange(10), np.arange(10)[1::-1]))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 0])

A quick and easy way to combine scalars and arrays is using ``np.r_``, with the demanded arrays, lists, or numbers in square brackets:

In [39]:
np.r_[2, 2, 2, np.arange(10), np.arange(10)[1::-1], [0, 1, 2]]

array([2, 2, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 0, 0, 1, 2])

``np.append`` uses concatenation internally:

In [37]:
np.append(np.arange(10), np.arange(10))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

For higher-dimensional arrays, other functions are useful:

In [36]:
np.stack((np.arange(10), np.arange(10)))

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

There are also the functions ``np.vstack`` (row-wise-stacking) and ``np.hstack`` (column-wise-stacking):
* hstack is equivalent to concatenation along the second axis, except for 1-D arrays where it concatenates along the first axis
* vstack is equivalent to concatenation along the first axis after 1-D arrays of shape (N,) have been reshaped to (1,N).

In [ ]:
two_dim_arr = np.arange(16).reshape(4, -1)
two_dim_arr_2 = np.arange(16).reshape(4, -1) + 16
two_dim_arr_2

In [ ]:
np.hstack((two_dim_arr, two_dim_arr_2))

In [ ]:
np.vstack((two_dim_arr, two_dim_arr_2))

### random.seed

If a random seed is set, the random-number-generator re-uses the same numbers over and over again. This is very useful for testing, but of course this takes any randomness out of anything, so it should not be used in final code:

In [76]:
for _ in range(5):
    np.random.seed(0)
    print(np.random.random(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]


To disable it, set a random seed of ``None``, in which case numpy generates random numbers using your system-randomness (or the system-time, ...)

In [77]:
for _ in range(5):
    np.random.seed(None)
    print(np.random.random(5))

[0.74061349 0.71302607 0.66900221 0.41957062 0.34863669]
[0.56720624 0.31873022 0.86192953 0.9897195  0.69954818]
[0.9250623  0.87770488 0.15464624 0.20509726 0.02965951]
[0.01383983 0.76803389 0.19909552 0.35890437 0.68433672]
[0.63403103 0.66755141 0.52648026 0.46259176 0.9336852 ]


### Shuffling arrays

``np.random.shuffle`` shuffles an array among the first index. That means, a one-dimensional is completely shuffled, whereas for multidimensional arrays, t

In [40]:
a = np.arange(10)
np.random.shuffle(a)
a

array([8, 7, 1, 4, 6, 2, 5, 0, 3, 9])

In [41]:
a = np.arange(9).reshape(3, 3)
np.random.shuffle(a)
a

array([[3, 4, 5],
       [0, 1, 2],
       [6, 7, 8]])

To shuffle the array completely, you can flatten it and afterwards reshape it to its original shape:

In [48]:
a = np.arange(9).reshape(3, 3)
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [64]:
b = a.flatten()
np.random.shuffle(b)
a = b.reshape(a.shape)
a

array([[2, 8, 7],
       [0, 4, 3],
       [6, 5, 1]])

Note that ``np.shuffle`` shuffles the array in-place. To return a permutation, you'd use ``np.permutation``:

In [65]:
a = np.arange(9).reshape(3, 3)
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [66]:
np.random.permutation(a.flatten()).reshape(a.shape)

array([[8, 6, 2],
       [1, 3, 0],
       [4, 7, 5]])

If you are dealing with different arrays you want to shuffle while keeping them matched to each other, it is more useful to shuffle the indices instead:

In [69]:
a = np.arange(9)+1
b = a**2
np.vstack((a, b))

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  4,  9, 16, 25, 36, 49, 64, 81]])

In [71]:
order = np.random.permutation(a.shape[0])
np.vstack((a[order], b[order]))

array([[ 9,  2,  4,  6,  5,  3,  7,  1,  8],
       [81,  4, 16, 36, 25,  9, 49,  1, 64]])

### Exercise 

Use numpy to generate a 5x5 array, where *precisely* 1/5 of the values are True

In [ ]:
count_down(3)

In [76]:
np.random.permutation(np.r_[np.ones(5, dtype=bool), np.zeros(20, dtype=bool)]).reshape(5,5)

array([[False, False,  True, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False,  True, False,  True],
       [False, False,  True, False, False]])

### np.random.choice

np.random.choice generates a sub-array from a given 1D-array:

In [144]:
np.random.seed(1)
a = np.arange(10)
np.random.choice(a, size=5)

array([5, 8, 9, 5, 0])

In [145]:
np.random.choice(a, size=5, replace=False)

array([6, 3, 9, 2, 8])

In [146]:
np.random.choice(a, size=11, replace=False)

ValueError: Cannot take a larger sample than population when 'replace=False'

You can also specify probabilities which which to take certain elements. To generate another array where roughly a quarter of elements are True, you can use:

In [148]:
np.random.choice(np.r_[True, False], size=(5,5), p=[0.25, 0.75])

array([[ True, False, False,  True, False],
       [False, False, False,  True,  True],
       [False,  True, False,  True, False],
       [ True, False, False, False, False],
       [False, False, False, False,  True]])

## Numpy print-options

In its standard options, numpy prints numbers only up to a certain accuracy, or consolidates the printing of arrays. ``np.set_printoptions`` can change that:

In [150]:
rand_arr = np.random.random((5,3))
rand_arr

array([[0.65229888, 0.36126102, 0.57100856],
       [0.63783648, 0.12631489, 0.69020459],
       [0.6477494 , 0.35393909, 0.76323305],
       [0.35653172, 0.75278835, 0.88134183],
       [0.01166919, 0.49810907, 0.07379201]])

In [151]:
np.set_printoptions(precision=3)
rand_arr

array([[0.652, 0.361, 0.571],
       [0.638, 0.126, 0.69 ],
       [0.648, 0.354, 0.763],
       [0.357, 0.753, 0.881],
       [0.012, 0.498, 0.074]])

In [152]:
rand_arr = rand_arr/1e3
rand_arr

array([[6.523e-04, 3.613e-04, 5.710e-04],
       [6.378e-04, 1.263e-04, 6.902e-04],
       [6.477e-04, 3.539e-04, 7.632e-04],
       [3.565e-04, 7.528e-04, 8.813e-04],
       [1.167e-05, 4.981e-04, 7.379e-05]])

In [155]:
np.set_printoptions(suppress=True, precision=6)
rand_arr

array([[0.000652, 0.000361, 0.000571],
       [0.000638, 0.000126, 0.00069 ],
       [0.000648, 0.000354, 0.000763],
       [0.000357, 0.000753, 0.000881],
       [0.000012, 0.000498, 0.000074]])

In [160]:
arr = np.arange(90)
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89])

In [161]:
np.set_printoptions(threshold=10)
arr

array([ 0,  1,  2, ..., 87, 88, 89])

## Further Readings

NumPy chapter from Jake VanderPlas's "Python Data Science Handbook" https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html

[Video tutorial from Scipy 2017](https://youtu.be/lKcwuPnSHIQ)


In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('lKcwuPnSHIQ')